https://github.com/graykode/nlp-tutorial 把他这个改成v2版本，并且学一下里面的模型  
Neural Network Language Model    
  
是通过前两个词来预测第三个词。其实就是把第三个字作为标签咯。这么想想nlp确实简单好学啊

In [1]:
# import tensorflow 确实会花去很多时间
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

In [ ]:
# tf.keras.backend.set_floatx('float64')  # 将模型的默认数据类型设置为float64, 原来为32

In [2]:
sentences = ["i like dog", "i love coffee", "i hate milk"]   # 这个就是全部数据了
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}   # word: index
number_dict = {i: w for i, w in enumerate(word_list)} # index: word
n_class = len(word_dict) # number of Vocabulary
print(number_dict)

{0: 'i', 1: 'dog', 2: 'hate', 3: 'coffee', 4: 'like', 5: 'love', 6: 'milk'}


In [3]:
# NNLM Parameter
n_step = 2 # number of steps ['i like', 'i love', 'i hate'] 这个step是啥意思啊？ 这是ngram
n_hidden = 2 # number of hidden units 隐藏层有2个神经元

# 准备训练batch，批处理数据
input_batch = []
target_batch = []

for sen in sentences:
    word = sen.split()
    input_ = [word_dict[n] for n in word[:-1]]
    print(input_)
    target = word_dict[word[-1]]
    print(target)
    # np.eye(n_class) 返回对角矩阵。一般独热码是这么生成的吗？
    input_batch.append(np.eye(n_class)[input_])
    target_batch.append(np.eye(n_class)[target])

[2, 1]
3
[2, 6]
5
[2, 4]
0


In [4]:
for sen in sentences:
    word = sen.split()
    print(word)
    print(word[:-1])

['i', 'like', 'dog']
['i', 'like']
['i', 'love', 'coffee']
['i', 'love']
['i', 'hate', 'milk']
['i', 'hate']


In [5]:
input_batch[0].dtype

NameError: name 'input_batch' is not defined

In [ ]:
input_batch

In [6]:
input_batch[0][0] + input_batch[0][1]

NameError: name 'input_batch' is not defined

In [7]:
target_batch

NameError: name 'target_batch' is not defined

In [8]:
class MyModel(Model):
    """
    子类化使用。更加高级
    """
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = Dense(2, activation='tanh')   # 二维卷积
        self.d2 = Dense(10, activation='softmax')

    def call(self, inputs, training=None, mask=None):
        inputs = self.d1(inputs)
        return self.d2(inputs)

In [9]:
# 生成模型
model = MyModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()  # 定义损失函数
optimizer = tf.keras.optimizers.Adam()   # 选择优化器


# metrics 观察训练过程
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')



In [10]:
# 将python的函数编译成tensorflow的图结构
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:  # 梯度带Record operations for automatic differentiation.
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [12]:
EPOCHS = 1
for epoch in range(EPOCHS):
    # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for i in range(len(sentences)):
        train_step(np.hstack((input_batch[i][0] + input_batch[i][1])), target_batch[i])

    for i in range(len(sentences)):
        test_step(np.hstack((input_batch[i][0] + input_batch[i][1])), target_batch[i])

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in converted code:

    <ipython-input-11-7579437bc139>:5 train_step  *
        predictions = model(images)
    /Users/ShayXU/anaconda/envs/python/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-9-852ad64d95ca>:11 call  *
        inputs = self.d1(inputs)
    /Users/ShayXU/anaconda/envs/python/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:812 __call__
        self.name)
    /Users/ShayXU/anaconda/envs/python/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/input_spec.py:193 assert_input_compatibility
        str(x.shape.as_list()))

    ValueError: Input 0 of layer dense is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: [7]
